# Programming Assignment: 
## Готовим LDA по рецептам

Как вы уже знаете, в тематическом моделировании делается предположение о том, что для определения тематики порядок слов в документе не важен; об этом гласит гипотеза «мешка слов». Сегодня мы будем работать с несколько нестандартной для тематического моделирования коллекцией, которую можно назвать «мешком ингредиентов», потому что на состоит из рецептов блюд разных кухонь. Тематические модели ищут слова, которые часто вместе встречаются в документах, и составляют из них темы. Мы попробуем применить эту идею к рецептам и найти кулинарные «темы». Эта коллекция хороша тем, что не требует предобработки. Кроме того, эта задача достаточно наглядно иллюстрирует принцип работы тематических моделей.

Для выполнения заданий, помимо часто используемых в курсе библиотек, потребуются модули *json* и *gensim*. Первый входит в дистрибутив Anaconda, второй можно поставить командой 

*pip install gensim*

Построение модели занимает некоторое время. На ноутбуке с процессором Intel Core i7 и тактовой частотой 2400 МГц на построение одной модели уходит менее 10 минут.

### Загрузка данных

Коллекция дана в json-формате: для каждого рецепта известны его id, кухня (cuisine) и список ингредиентов, в него входящих. Загрузить данные можно с помощью модуля json (он входит в дистрибутив Anaconda):

In [2]:
import json

In [3]:
with open("recipes.json") as f:
    recipes = json.load(f)

In [4]:
print recipes[0]

{u'cuisine': u'greek', u'id': 10259, u'ingredients': [u'romaine lettuce', u'black olives', u'grape tomatoes', u'garlic', u'pepper', u'purple onion', u'seasoning', u'garbanzo beans', u'feta cheese crumbles']}


### Составление корпуса

In [81]:
from gensim import corpora, models
from gensim.models import LdaModel
import numpy as np
import pandas as pd

Наша коллекция небольшая, и целиком помещается в оперативную память. Gensim может работать с такими данными и не требует их сохранения на диск в специальном формате. Для этого коллекция должна быть представлена в виде списка списков, каждый внутренний список соответствует отдельному документу и состоит из его слов. Пример коллекции из двух документов: 

[["hello", "world"], ["programming", "in", "python"]]

Преобразуем наши данные в такой формат, а затем создадим объекты corpus и dictionary, с которыми будет работать модель.

In [6]:
texts = [recipe["ingredients"] for recipe in recipes]
dictionary = corpora.Dictionary(texts)   # составляем словарь
corpus = [dictionary.doc2bow(text) for text in texts]  # составляем корпус документов

In [7]:
print texts[0]
print corpus[0]
print len(texts)

[u'romaine lettuce', u'black olives', u'grape tomatoes', u'garlic', u'pepper', u'purple onion', u'seasoning', u'garbanzo beans', u'feta cheese crumbles']
[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1)]
39774


У объекта dictionary есть полезная переменная dictionary.token2id, позволяющая находить соответствие между ингредиентами и их индексами.

### Обучение модели
Вам может понадобиться [документация](https://radimrehurek.com/gensim/models/ldamodel.html) LDA в gensim.

__Задание 1.__ Обучите модель LDA с 40 темами, установив количество проходов по коллекции 5 и оставив остальные параметры по умолчанию. 


Затем вызовите метод модели *show_topics*, указав количество тем 40 и количество токенов 10, и сохраните результат (топы ингредиентов в темах) в отдельную переменную. Если при вызове метода *show_topics* указать параметр *formatted=True*, то топы ингредиентов будет удобно выводить на печать, если *formatted=False*, будет удобно работать со списком программно. Выведите топы на печать, рассмотрите темы, а затем ответьте на вопрос:

Сколько раз ингредиенты "salt", "sugar", "water", "mushrooms", "chicken", "eggs" встретились среди топов-10 всех 40 тем? При ответе __не нужно__ учитывать составные ингредиенты, например, "hot water".

Передайте 6 чисел в функцию save_answers1 и загрузите сгенерированный файл в форму.

У gensim нет возможности фиксировать случайное приближение через параметры метода, но библиотека использует numpy для инициализации матриц. Поэтому, по утверждению автора библиотеки, фиксировать случайное приближение нужно командой, которая написана в следующей ячейке. __Перед строкой кода с построением модели обязательно вставляйте указанную строку фиксации random.seed.__

In [8]:
np.random.seed(76543)

lda = LdaModel(corpus=corpus, id2word=dictionary, num_topics=40, passes=5)
topics = lda.show_topics(num_topics=40, num_words=10, formatted=True)

In [9]:
checked_list = ["salt", "sugar", "water", "mushrooms", "chicken", "eggs"]
amounts = dict() 

id_checked_list = []
dic = zip(*dictionary.items())
for i in checked_list:
    id_ = dic[1].index(i)
    id_checked_list.append(i)
    amounts[i] = 0
    
print id_checked_list
print amounts

for i in topics:
    for j in id_checked_list:
        sr = '"' + j + '"'
#         print i[1]
        if i[1].find(sr) > 0:
            amounts[j] += 1
            
print amounts

['salt', 'sugar', 'water', 'mushrooms', 'chicken', 'eggs']
{'mushrooms': 0, 'eggs': 0, 'sugar': 0, 'water': 0, 'chicken': 0, 'salt': 0}
{'mushrooms': 1, 'eggs': 1, 'sugar': 7, 'water': 8, 'chicken': 1, 'salt': 20}


In [12]:
save_answers1(amounts['salt'], amounts['sugar'], amounts['water'], amounts['mushrooms'], amounts['chicken'], amounts['eggs'])

In [11]:
def save_answers1(c_salt, c_sugar, c_water, c_mushrooms, c_chicken, c_eggs):
    with open("cooking_LDA_pa_task1.txt", "w") as fout:
        fout.write(" ".join([str(el) for el in [c_salt, c_sugar, c_water, c_mushrooms, c_chicken, c_eggs]]))

### Фильтрация словаря
В топах тем гораздо чаще встречаются первые три рассмотренных ингредиента, чем последние три. При этом наличие в рецепте курицы, яиц и грибов яснее дает понять, что мы будем готовить, чем наличие соли, сахара и воды. Таким образом, даже в рецептах есть слова, часто встречающиеся в текстах и не несущие смысловой нагрузки, и поэтому их не желательно видеть в темах. Наиболее простой прием борьбы с такими фоновыми элементами — фильтрация словаря по частоте. Обычно словарь фильтруют с двух сторон: убирают очень редкие слова (в целях экономии памяти) и очень частые слова (в целях повышения интерпретируемости тем). Мы уберем только частые слова.

In [13]:
import copy
dictionary2 = copy.deepcopy(dictionary)

__Задание 2.__ У объекта dictionary2 есть переменная *dfs* — это словарь, ключами которого являются id токена, а элементами — число раз, сколько слово встретилось во всей коллекции. Сохраните в отдельный список ингредиенты, которые встретились в коллекции больше 4000 раз. Вызовите метод словаря *filter_tokens*, подав в качестве первого аргумента полученный список популярных ингредиентов. Вычислите две величины: dict_size_before и dict_size_after — размер словаря до и после фильтрации.

Затем, используя новый словарь, создайте новый корпус документов, corpus2, по аналогии с тем, как это сделано в начале ноутбука. Вычислите две величины: corpus_size_before и corpus_size_after — суммарное количество ингредиентов в корпусе (для каждого документа вычислите число различных ингредиентов в нем и просуммируйте по всем документам) до и после фильтрации.

Передайте величины dict_size_before, dict_size_after, corpus_size_before, corpus_size_after в функцию save_answers2 и загрузите сгенерированный файл в форму.

In [14]:
new_d = {}
dict_size_before = len(dictionary2.dfs.items())

for k in dictionary2.dfs.keys():
    if dictionary2.dfs[k] > 4000:
        new_d[k] = dictionary2.dfs[k]
        
print new_d
list_of_bad = list(new_d)

dictionary2.filter_tokens(list_of_bad)
dict_size_after = len(dictionary2.dfs.items())

print dict_size_before
print dict_size_after

{0: 4438, 4: 7380, 9: 4385, 12: 4784, 45: 7972, 48: 6434, 17: 18048, 21: 4847, 54: 7971, 100: 6236, 29: 7457, 117: 4632}
6714
6702


In [15]:
def size_corpus(corpus):
    l = 0
    for i in range(len(corpus)):
        l += len(corpus[i])
    return l

corpus2 = [dictionary2.doc2bow(text) for text in texts]
corpus_size_before = size_corpus(corpus)
corpus_size_after = size_corpus(corpus2)
print corpus_size_before
print corpus_size_after

428249
343665


In [18]:
save_answers2(dict_size_before, dict_size_after, corpus_size_before, corpus_size_after)

In [17]:
def save_answers2(dict_size_before, dict_size_after, corpus_size_before, corpus_size_after):
    with open("cooking_LDA_pa_task2.txt", "w") as fout:
        fout.write(" ".join([str(el) for el in [dict_size_before, dict_size_after, corpus_size_before, corpus_size_after]]))

### Сравнение когерентностей
__Задание 3.__ Постройте еще одну модель по корпусу corpus2 и словарю dictionary2, остальные параметры оставьте такими же, как при первом построении модели. Сохраните новую модель в другую переменную (не перезаписывайте предыдущую модель). Не забудьте про фиксирование seed!

Затем воспользуйтесь методом *top_topics* модели, чтобы вычислить ее когерентность. Передайте в качестве аргумента соответствующий модели корпус. Метод вернет список кортежей (топ токенов, когерентность), отсортированных по убыванию последней. Вычислите среднюю по всем темам когерентность для каждой из двух моделей и передайте в функцию save_answers3. 

In [19]:
np.random.seed(76543)

lda2 = LdaModel(corpus=corpus2, id2word=dictionary2, num_topics=40, passes=5)
top_topics_1 = lda.top_topics(corpus)
top_topics_2 = lda2.top_topics(corpus2)

In [21]:
dic__1 = zip(*top_topics_1)
dic__2 = zip(*top_topics_2)

coherence = np.asarray(dic__1[1]).mean()
coherence2 = np.asarray(dic__2[1]).mean()

In [22]:
import gensim
print gensim.__version__
save_answers3(coherence, coherence2)

2.3.0


In [20]:
def save_answers3(coherence, coherence2):
    with open("cooking_LDA_pa_task3.txt", "w") as fout:
        fout.write(" ".join(["%3f"%el for el in [coherence, coherence2]]))

Считается, что когерентность хорошо соотносится с человеческими оценками интерпретируемости тем. Поэтому на больших текстовых коллекциях когерентность обычно повышается, если убрать фоновую лексику. Однако в нашем случае этого не произошло. 

### Изучение влияния гиперпараметра alpha

В этом разделе мы будем работать со второй моделью, то есть той, которая построена по сокращенному корпусу. 

Пока что мы посмотрели только на матрицу темы-слова, теперь давайте посмотрим на матрицу темы-документы. Выведите темы для нулевого (или любого другого) документа из корпуса, воспользовавшись методом *get_document_topics* второй модели:

In [23]:
print lda2.get_document_topics(bow=corpus2)[0]

[(4, 0.12812499999999991), (15, 0.32368464962559051), (22, 0.27665871617377874), (37, 0.15903163420063021)]


Также выведите содержимое переменной *.alpha* второй модели:

In [24]:
print lda2.alpha

[ 0.025  0.025  0.025  0.025  0.025  0.025  0.025  0.025  0.025  0.025
  0.025  0.025  0.025  0.025  0.025  0.025  0.025  0.025  0.025  0.025
  0.025  0.025  0.025  0.025  0.025  0.025  0.025  0.025  0.025  0.025
  0.025  0.025  0.025  0.025  0.025  0.025  0.025  0.025  0.025  0.025]


У вас должно получиться, что документ характеризуется небольшим числом тем. Попробуем поменять гиперпараметр alpha, задающий априорное распределение Дирихле для распределений тем в документах.

__Задание 4.__ Обучите третью модель: используйте сокращенный корпус (corpus2 и dictionary2) и установите параметр __alpha=1__, passes=5. Не забудьте про фиксацию seed! Выведите темы новой модели для нулевого документа; должно получиться, что распределение над множеством тем практически равномерное. Чтобы убедиться в том, что во второй модели документы описываются гораздо более разреженными распределениями, чем в третьей, посчитайте суммарное количество элементов, __превосходящих 0.01__, в матрицах темы-документы обеих моделей. Другими словами, запросите темы  модели для каждого документа с параметром *minimum_probability=0.01* и просуммируйте число элементов в получаемых массивах. Передайте две суммы (сначала для модели с alpha по умолчанию, затем для модели в alpha=1) в функцию save_answers4.

In [25]:
np.random.seed(76543)

lda3 = LdaModel(corpus=corpus2, id2word=dictionary2, num_topics=40, passes=5, alpha=1)
# print lda3.get_document_topics(bow=corpus2)[0]

[(0, 0.021286343792953501), (1, 0.021298590310099758), (2, 0.02132005550857179), (3, 0.021282109727077194), (4, 0.021276595744680844), (5, 0.021496606933716376), (6, 0.021292176774168874), (7, 0.04255970642916266), (8, 0.02127663405745726), (9, 0.021276595744680844), (10, 0.021574647158752062), (11, 0.021287570547940532), (12, 0.021277424425023877), (13, 0.084096705567749536), (14, 0.021290106135531343), (15, 0.021695610486780717), (16, 0.021284453838081592), (17, 0.0212800707131021), (18, 0.02134003909164137), (19, 0.021278463140557358), (20, 0.021276595744752807), (21, 0.021334086322766982), (22, 0.021281172160394983), (23, 0.021394919949490786), (24, 0.021288024651372802), (25, 0.062709262165435847), (26, 0.021292249798166074), (27, 0.021292331136060234), (28, 0.021307418247128992), (29, 0.021434803424634743), (30, 0.021432324163326067), (31, 0.021282086472031888), (32, 0.021325744739054311), (33, 0.042881410740644124), (34, 0.021276595744680955), (35, 0.021282060707513067), (36, 0.

In [65]:
def sum_by_topics(topic):
    sum = 0
    for i in topic:
        sum += len(topic)
    return sum

# lda2.minimum_probability = 0.01
# lda3.minimum_probability = 0.01
topics2 = lda2.get_document_topics(bow=corpus2, minimum_probability=0.01)
topics3 = lda3.get_document_topics(bow=corpus2, minimum_probability=0.01)

sum_2 = sum_by_topics(topics2)
sum_3 = sum_by_topics(topics3)

In [30]:
save_answers4(sum_2, sum_3)

In [27]:
def save_answers4(count_model2, count_model3):
    with open("cooking_LDA_pa_task4.txt", "w") as fout:
        fout.write(" ".join([str(el) for el in [count_model2, count_model3]]))

Таким образом, гиперпараметр __alpha__ влияет на разреженность распределений тем в документах. Аналогично гиперпараметр __eta__ влияет на разреженность распределений слов в темах.

### LDA как способ понижения размерности
Иногда, распределения над темами, найденные с помощью LDA, добавляют в матрицу объекты-признаки как дополнительные, семантические, признаки, и это может улучшить качество решения задачи. Для простоты давайте просто обучим классификатор рецептов на кухни на признаках, полученных из LDA, и измерим точность (accuracy).

__Задание 5.__ Используйте модель, построенную по сокращенной выборке с alpha по умолчанию (вторую модель). Составьте матрицу $\Theta = p(t|d)$ вероятностей тем в документах; вы можете использовать тот же метод get_document_topics, а также вектор правильных ответов y (в том же порядке, в котором рецепты идут в переменной recipes). Создайте объект RandomForestClassifier со 100 деревьями, с помощью функции cross_val_score вычислите среднюю accuracy по трем фолдам (перемешивать данные не нужно) и передайте в функцию save_answers5.

In [31]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_score

/home/toxa/anaconda3/envs/py27/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [73]:
X = np.ndarray(shape=(len(texts), 40))
for i, t2 in enumerate(topics2):
    for k, t2_i in enumerate(topics2[i]):
        X[i][t2_i[0]] = t2_i[1]

(39774, 40)


In [89]:
print recipes[0]['cuisine']
cuisine_names = []
for i,v in enumerate(recipes):  
    cuisine_names.append(v['cuisine'])
y = pd.get_dummies(cuisine_names)
print y.shape

greek
(39774, 20)


In [90]:
np.random.seed(76543)

acc = cross_val_score(estimator=RandomForestClassifier(n_estimators=100, random_state=0),X=X,y=y,cv=3)
accuracy_mean_score = np.mean(acc)
print accuracy_mean_score

save_answers5(accuracy_mean_score)

0.395233066828


In [74]:
def save_answers5(accuracy):
     with open("cooking_LDA_pa_task5.txt", "w") as fout:
        fout.write(str(accuracy))

Для такого большого количества классов это неплохая точность. Вы можете попроовать обучать RandomForest на исходной матрице частот слов, имеющей значительно большую размерность, и увидеть, что accuracy увеличивается на 10–15%. Таким образом, LDA собрал не всю, но достаточно большую часть информации из выборки, в матрице низкого ранга.

### LDA — вероятностная модель
Матричное разложение, использующееся в LDA, интерпретируется как следующий процесс генерации документов.

Для документа $d$ длины $n_d$:
1. Из априорного распределения Дирихле с параметром alpha сгенерировать распределение над множеством тем: $\theta_d \sim Dirichlet(\alpha)$
1. Для каждого слова $w = 1, \dots, n_d$:
    1. Сгенерировать тему из дискретного распределения $t \sim \theta_{d}$
    1. Сгенерировать слово из дискретного распределения $w \sim \phi_{t}$.
    
Подробнее об этом в [Википедии](https://en.wikipedia.org/wiki/Latent_Dirichlet_allocation).

В контексте нашей задачи получается, что, используя данный генеративный процесс, можно создавать новые рецепты. Вы можете передать в функцию модель и число ингредиентов и сгенерировать рецепт :)

In [ ]:
def generate_recipe(model, num_ingredients):
    theta = np.random.dirichlet(model.alpha)
    for i in range(num_ingredients):
        t = np.random.choice(np.arange(model.num_topics), p=theta)
        topic = model.show_topic(t, topn=model.num_terms)
        topic_distr = [x[1] for x in topic]
        terms = [x[0] for x in topic]
        w = np.random.choice(terms, p=topic_distr)
        print w

### Интерпретация построенной модели
Вы можете рассмотреть топы ингредиентов каждой темы. Большиснтво тем сами по себе похожи на рецепты; в некоторых собираются продукты одного вида, например, свежие фрукты или разные виды сыра.

Попробуем эмпирически соотнести наши темы с национальными кухнями (cuisine). Построим матрицу $A$ размера темы $x$ кухни, ее элементы $a_{tc}$ — суммы $p(t|d)$ по всем документам $d$, которые отнесены к кухне $c$. Нормируем матрицу на частоты рецептов по разным кухням, чтобы избежать дисбаланса между кухнями. Следующая функция получает на вход объект модели, объект корпуса и исходные данные и возвращает нормированную матрицу $A$. Ее удобно визуализировать с помощью seaborn.

In [ ]:
import pandas
import seaborn
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
def compute_topic_cuisine_matrix(model, corpus, recipes):
    # составляем вектор целевых признаков
    targets = list(set([recipe["cuisine"] for recipe in recipes]))
    # составляем матрицу
    tc_matrix = pandas.DataFrame(data=np.zeros((model.num_topics, len(targets))), columns=targets)
    for recipe, bow in zip(recipes, corpus):
        recipe_topic = model.get_document_topics(bow)
        for t, prob in recipe_topic:
            tc_matrix[recipe["cuisine"]][t] += prob
    # нормируем матрицу
    target_sums = pandas.DataFrame(data=np.zeros((1, len(targets))), columns=targets)
    for recipe in recipes:
        target_sums[recipe["cuisine"]] += 1
    return pandas.DataFrame(tc_matrix.values/target_sums.values, columns=tc_matrix.columns)

In [ ]:
def plot_matrix(tc_matrix):
    plt.figure(figsize=(10, 10))
    seaborn.heatmap(tc_matrix, square=True)

In [ ]:
# Визуализируйте матрицу


Чем темнее квадрат в матрице, тем больше связь этой темы с данной кухней. Мы видим, что у нас есть темы, которые связаны с несколькими кухнями. Такие темы показывают набор ингредиентов, которые популярны в кухнях нескольких народов, то есть указывают на схожесть кухонь этих народов. Некоторые темы распределены по всем кухням равномерно, они показывают наборы продуктов, которые часто используются в кулинарии всех стран. 

Жаль, что в датасете нет названий рецептов, иначе темы было бы проще интерпретировать...

### Заключение
В этом задании вы построили несколько моделей LDA, посмотрели, на что влияют гиперпараметры модели и как можно использовать построенную модель. 